In [23]:
# 1. Standard imports
import pandas as pd
import numpy as np
from datetime import timedelta

# 2. Load the transactions dataset
transactions = pd.read_csv('../data/transactions.csv')

# Group by customer and calculate mean and std dev of transaction amounts
customer_amount_stats = transactions.groupby('customer_id')['amount'].agg(['mean', 'std']).reset_index()
customer_amount_stats.rename(columns={'mean': 'amount_mean', 'std': 'amount_std'}, inplace=True)

# Merge stats back onto transactions
transactions = transactions.merge(customer_amount_stats, on='customer_id', how='left')


# Avoid divide-by-zero by replacing std = 0 with np.nan, Calculate z-score then fill z-score with 0 if na
transactions['amount_std'] = transactions['amount_std'].replace(0, np.nan)
transactions['amount_zscore'] = (transactions['amount'] - transactions['amount_mean']) / transactions['amount_std']
transactions['amount_zscore'] = transactions['amount_zscore'].fillna(0)



# Step 1: Ensure timestamp is datetime
transactions['timestamp'] = pd.to_datetime(transactions['timestamp'])
                                           
# Step 2: Sort by customer and time
transactions.sort_values(by=['customer_id', 'timestamp'], inplace=True)

# Step 3: Define a function to apply per-customer
def compute_txn_count_last_7d(group):
    # Empty list to collect counts
    counts = []
    for i, current_row in group.iterrows():
        # Filter: prior 7-day window
        start_time = current_row['timestamp'] - timedelta(days=7)
        mask = (group['timestamp'] >= start_time) & (group['timestamp'] < current_row['timestamp'])
        count = mask.sum()
        counts.append(count)
    group['txn_count_last_7d'] = counts
    return group

# Step 4: Apply to each customer
transactions = (
    transactions
    .groupby('customer_id', group_keys=False)
    .apply(compute_txn_count_last_7d)
    .reset_index(drop=True)
)

def compute_amount_ratio_last_30d(group):
    ratios = []
    for i, current_row in group.iterrows():
        # Define 30-day lookback window
        start_time = current_row['timestamp'] - timedelta(days=30)
        mask = (group['timestamp'] >= start_time) & (group['timestamp'] < current_row['timestamp'])
        prior_txns = group[mask]

        if len(prior_txns) > 0:
            avg_amount = prior_txns['amount'].mean()
            ratio = current_row['amount'] / avg_amount if avg_amount != 0 else np.nan
        else:
            ratio = 1.0 # treat no history as neutral
        ratios.append(ratio)
    
    group['amount_to_avg_amount_30d'] = ratios
    return group

transactions = (
    transactions
    .groupby('customer_id', group_keys=False)
    .apply(compute_amount_ratio_last_30d)
    .reset_index(drop=True)
)

transactions.head()

C:\Users\tyler\AppData\Local\Temp\ipykernel_17240\3602198546.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_txn_count_last_7d)
C:\Users\tyler\AppData\Local\Temp\ipykernel_17240\3602198546.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_amount_ratio_last_30d)


,transaction_id,timestamp,location,amount,customer_id,merchant_id,fraud,amount_mean,amount_std,amount_zscore,txn_count_last_7d,amount_to_avg_amount_30d
0,5a0b98cb-de93-44f4-9728-23690a0b8258,2024-05-03 22:37:05,Connieshire,3797.60,005a28cf-c6ce-4fff-ad4a-a2b358e5b400,acf225b1-ab1a-4027-8517-cca814c14e83,0,2537.183804,1349.968439,0.933663,0,1.000000
1,a2debdd1-4214-4ff6-a476-f277f500043b,2024-05-06 20:53:33,Kelleyshire,2386.53,005a28cf-c6ce-4fff-ad4a-a2b358e5b400,6dbc8ab9-13f7-4ad5-92fc-599bbb5da13e,0,2537.183804,1349.968439,-0.111598,1,0.628431
2,e2929746-5011-451e-9e73-afa8facc7781,2024-05-08 01:47:39,South Derrickfort,3484.63,005a28cf-c6ce-4fff-ad4a-a2b358e5b400,dd7d3ba3-8570-4f26-b09e-221cc44c095f,0,2537.183804,1349.968439,0.701828,2,1.126959
3,075e3390-f507-4b26-9f32-b64deea4708e,2024-05-10 12:25:19,Jennifershire,2426.54,005a28cf-c6ce-4fff-ad4a-a2b358e5b400,ef9abb83-5675-4b3a-926b-4d686894969d,0,2537.183804,1349.968439,-0.081960,3,0.752901
4,7857c287-a579-4f14-9075-4acfbc8bd5e0,2024-05-12 23:17:25,East Waynestad,157.35,005a28cf-c6ce-4fff-ad4a-a2b358e5b400,84655063-250b-44e6-a5da-a2715e9f5a5c,0,2537.183804,1349.968439,-1.762881,3,0.052037


In [6]:
nan = transactions.loc[transactions['tran_count']==1]
nan

,transaction_id,timestamp,location,amount,customer_id,merchant_id,fraud,amount_mean,amount_std,amount_zscore,tran_count
